In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
#! Set these variables
work_dir_name = "pituitary3"

In [5]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = drive_root / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [6]:
labels = ["pituitary"]
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="flair.t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(labels, suffix_list=["CH", "SRS", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-12 19:11:34.533 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/10 [00:00<?, ?it/s]2024-12-12 19:11:34.574 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pituitary-CH.nii.gz for Scan(subid=1010, sesid=20180208)
2024-12-12 19:11:34.589 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pituitary-CH.nii.gz for Scan(subid=1033, sesid=20171117)
2024-12-12 19:11:34.602 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pituitary-CH.nii.gz for Scan(subid=1119, sesid=20161010)
2024-12-12 19:11:34.614 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pituitary-CH.nii.gz for Scan(subid=1259, sesid=20200803)
2024-12-12 19:11:34.626 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pituitary-CH.nii.gz for Scan(subid=1437, sesid=20210503)
2024-12-12 19:11:34.638 | INFO     | monai_training.preprocess:prepare_labels:226 - Found label pi

Calculate Dice Score for Pituitary

In [7]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "pituitary", ["CH", "SRS", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=1)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.81
1033: 0.82
1119: 0.88
1259: 0.89
1437: 0.88
1547: 0.85
2081: 0.81
2126: 0.83
2146: 0.83
2187: 0.83
----------
Mean: 0.84
Std: 0.03
[0.8089528377298162, 0.8212703101920237, 0.8764890282131661, 0.8885704048728055, 0.8763368983957219, 0.8458523840627041, 0.8128772635814889, 0.8256410256410256, 0.829198130169004, 0.8300884955752212]


Calculate Hausdorff Distance for Pituitary

In [8]:
haus = []
print("Hausdorff Dist")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "pituitary", ["CH", "SRS", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

Hausdorff Dist
1010: 2.00
1033: 1.41
1119: 1.41
1259: 1.00
1437: 1.41
1547: 2.00
2081: 2.00
2126: 1.41
2146: 1.73
2187: 1.41
----------
Mean: 1.58
Std: 0.34
[2.0, 1.4142135623730951, 1.4142135623730951, 1.0, 1.4142135623730951, 2.0, 2.0, 1.4142135623730951, 1.7320508075688772, 1.4142135623730951]
